This file is used for training a YOLOv5 Model.

# Step 1 Preparations

## Import Packages

In [2]:
!pip install torch

^C


In [1]:


import torch
import os
import glob
from IPython.display import Image, clear_output  # to display images

%pip install -q roboflow
from roboflow import Roboflow

Note: you may need to restart the kernel to use updated packages.


## Direct to your Google Drive

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/YOLO')

Mounted at /content/drive


## Clone YOLOv5 (optional if you have done it before)

In [ ]:
# clone YOLOv5 and
!git clone https://github.com/dqanxy-umich/yolov5-infrared-capability.git  # clone repo
%cd yolov5-infrared-capability
%pip install -qr requirements.txt # install dependencies

fatal: destination path 'yolov5-infrared-capability' already exists and is not an empty directory.
/content/yolov5-infrared-capability
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.7/881.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00


## Complete the Setup

In [ ]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.4.1+cu121 (CPU)


# Step 2 Load the Dataset

## Copy and paste the dataset download codes.

In [2]:
%cd C:/Users/dt800/Downloads/YOLO-20241004T055338Z-001/YOLO/yolov5-infrared-capability/datasets

rf = Roboflow(api_key="lbtWuCKm7wXxOrtP5zn6")
project = rf.workspace("mdp-0gchk").project("pwas")
version = project.version(34)
dataset = version.download("yolov5")


C:\Users\dt800\Downloads\YOLO-20241004T055338Z-001\YOLO\yolov5-infrared-capability\datasets
loading Roboflow workspace...


This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.


loading Roboflow project...


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
%cd C:/Users/dt800/Downloads/YOLO-20241004T055338Z-001/YOLO/yolov5-infrared-capability
!python train.py --ch 4 --batch 16 --epochs 200 --cfg yolov5s.yaml --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

C:\Users\dt800\Downloads\YOLO-20241004T055338Z-001\YOLO\yolov5-infrared-capability


# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

KeyboardInterrupt: 

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# make sure to change the filepath of the trained weight
!python detect.py --weights runs/train/exp6/weights/best.pt --conf 0.2 --source {dataset.location}/test/images

detect: weights=['runs/train/exp6/weights/best.pt'], source=/content/drive/MyDrive/YOLO/yolov5-infrared-capability/datasets/PWAS-34/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-298-ga5860f62 Python-3.10.12 torch-2.4.1+cu121 CPU

Traceback (most recent call last):
  File "/content/drive/MyDrive/YOLO/yolov5-infrared-capability/detect.py", line 312, in <module>
    main(opt)
  File "/content/drive/MyDrive/YOLO/yolov5-infrared-capability/detect.py", line 307, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return f

In [ ]:
#display inference on ALL test images
for imageName in glob.glob('/content/drive/MyDrive/YOLO/yolov5/runs/detect/exp4/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Conclusion and Next Steps

You've trained a custom YOLOv5 model to recognize your custom objects.

To improve the model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

## Download the best weight file.

In [ ]:
#export your model's weights for future use
files.download('./runs/train/exp6/weights/best.pt') #remember to change the path

## **Upload the trained weight file to Roboflow.**

In [ ]:
version.deploy("yolov5", "/content/drive/MyDrive/YOLO/yolov5/runs/train/exp6") #remember to change the path